In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta
import os
import time

In [ ]:
# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# Check if we're using GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Using device: cuda
GPU: NVIDIA A100-SXM4-40GB


In [ ]:
# 2. Data Loading and Preparation
# 2.1 Load Input Data
# Load the data (modify paths as needed, data is shared here: https://drive.google.com/drive/u/0/folders/1ORxHegaYo2Clufvce78shuwMXyTt4epJ)
hourly_data_csv = 'hourly_data.csv'
hourly_ontario_csv = 'hourly_ontario_demand.csv'
forecast_climate_csv = 'forecast_climate.csv'
actual_toronto_demand_csv = 'actual_toronto_demand.csv'
actual_ontario_demand_csv = 'actual_ontario_demand.csv'

In [ ]:
hourly_data = pd.read_csv(hourly_data_csv)
hourly_data['datetime'] = pd.to_datetime(hourly_data['datetime'])

hourly_ontario = pd.read_csv(hourly_ontario_csv)
hourly_ontario['datetime'] = pd.to_datetime(hourly_ontario['datetime'])

forecast_climate = pd.read_csv(forecast_climate_csv)
forecast_climate['datetime'] = pd.to_datetime(forecast_climate['datetime'])

actual_toronto_demand = pd.read_csv(actual_toronto_demand_csv)
actual_toronto_demand = actual_toronto_demand.rename(columns={"Toronto": 'actual'})
actual_toronto_demand['datetime'] = pd.to_datetime(actual_toronto_demand['datetime'])

actual_ontario_demand = pd.read_csv(actual_ontario_demand_csv)
actual_ontario_demand['datetime'] = pd.to_datetime(actual_ontario_demand['datetime'])
actual_ontario_demand = actual_ontario_demand.rename(columns={"Ontario Demand": 'actual'})

In [ ]:
# 3. Dataset Definition
class TimeSeriesDataset(Dataset):
    """PyTorch Dataset for time series data"""
    def __init__(self, features, targets):
        self.features = torch.tensor(features, dtype=torch.float32)
        # Ensure targets have shape [n_samples, 1]
        if len(targets.shape) == 1:
            targets = targets.reshape(-1, 1)
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

In [ ]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super(MultiHeadSelfAttention, self).__init__()
        self.multihead_attn = nn.MultiheadAttention(embed_dim, num_heads, batch_first=True)

    def forward(self, x):
        # x shape: (batch_size, seq_len, embed_dim)
        attn_output, _ = self.multihead_attn(x, x, x)
        return attn_output

In [ ]:
class LSTMAttentionModel(nn.Module):
    def __init__(self, input_shape, hidden_dim=128, lstm_layers=2, dropout=0.3, output_shape=1):
        super(LSTMAttentionModel, self).__init__()
        self.seq_length, self.n_features = input_shape
        self.hidden_dim = hidden_dim
        self.lstm_layers = lstm_layers

        # LSTM layers for sequence processing
        self.lstm = nn.LSTM(
            input_size=self.n_features,
            hidden_size=hidden_dim,
            num_layers=lstm_layers,
            dropout=dropout if lstm_layers > 1 else 0,
            batch_first=True
        )

        # Attention mechanism
        self.attention = MultiHeadSelfAttention(embed_dim=hidden_dim, num_heads=4)
        self.layer_norm = nn.LayerNorm(hidden_dim)

        # Global pooling
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)

        # Fully connected layers
        self.fc1 = nn.Linear(hidden_dim, hidden_dim)
        self.dropout1 = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_dim, 64)
        self.dropout2 = nn.Dropout(dropout)
        self.fc3 = nn.Linear(64, output_shape)

    def forward(self, x):
        # x shape: (batch_size, seq_length, n_features)

        # LSTM layers
        lstm_out, _ = self.lstm(x)
        # lstm_out shape: (batch_size, seq_length, hidden_dim)

        # Apply attention
        attn_output = self.attention(lstm_out)

        # Skip connection and layer normalization
        x = lstm_out + attn_output
        x = self.layer_norm(x)

        # Global average pooling across sequence dimension
        x = x.transpose(1, 2)  # Shape: (batch_size, hidden_dim, seq_length)
        x = self.global_avg_pool(x).squeeze(-1)  # Shape: (batch_size, hidden_dim)

        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)

        # Reshape to ensure output has shape [batch_size, 1]
        x = x.view(-1, 1)

        return x

In [ ]:
# 6. Data Processing and Model Training Functions
# 6.1 Data Preparation Functions
def prepare_data_for_model(region, data, target_column, window_size=168, forecast_horizon=1):
    """Prepare time series data for the LSTM-Attention model"""
    
     # Extract target variable
    y_values = data[target_column].values

    # Get all feature columns (excluding target and datetime columns)
    exclude_cols = ['datetime', 'date', target_column, 'region']
    # Select only numeric columns
    numeric_columns = data.select_dtypes(include=['number']).columns
    feature_columns = [col for col in numeric_columns if col not in exclude_cols]
    print(f"Feature columns count: {len(feature_columns)}")

    num_sliding_windows = len(data) - window_size - forecast_horizon + 1
    print(f"Number of sliding windows: {num_sliding_windows}")

    report_frequency = max(1, num_sliding_windows // 20)
    start_time = time.time()

    # Create sequences using sliding window approach
    X, y = [], []
    for i in range(num_sliding_windows):
        # Extract the window of features
        features = data[feature_columns].iloc[i:(i+window_size)].values

        # Extract the target value(s) to predict
        target = y_values[i + window_size:i + window_size + forecast_horizon]

        X.append(features)
        y.append(target)

        # Report progress
        if i % report_frequency == 0:
            percentage = round((i / num_sliding_windows) * 100, 2)
            elapsed_time = time.time() - start_time
            print(f"Processed {i}/{num_sliding_windows} windows, ({percentage}%) in {elapsed_time:.2f} seconds")

    X = np.array(X)
    y = np.array(y)
    print(f"Data preparation complete: X shape: {X.shape}, y shape: {y.shape}")

    return X, y

In [ ]:
def scale_features(X, y):
  """Scale features and targets using StandardScaler"""
  # Scale features
  n_samples, n_timesteps, n_features = X.shape
  X_reshaped = X.reshape(n_samples * n_timesteps, n_features)

  scaler_X = StandardScaler()
  scaler_X.fit(X_reshaped)
  X_scaled = scaler_X.transform(X_reshaped).reshape(n_samples, n_timesteps, n_features)

  scaler_y = StandardScaler()
  scaler_y.fit(y)
  y_scaled = scaler_y.transform(y).flatten()

  return X_scaled, y_scaled

# 6.2 Model Training Functions

In [ ]:
def train_model(model, loader, device,
               epochs=50, learning_rate=0.001, model_save_path='best_lstm_model.pth'):
    """Train the PyTorch model with early stopping"""
    # Move model to device (GPU if available)
    model = model.to(device)

    # Loss and optimizer
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # For tracking metrics
    history = {
        'loss': [],
        'mae': []
    }

    # Training loop
    for epoch in range(epochs):
        # Training phase
        model.train()
        losses = []
        maes = []

        print(f"\nEpoch {epoch+1}/{epochs}:")

        # Training loop
        for batch_idx, (inputs, targets) in enumerate(loader):
            # Move data to device
            inputs, targets = inputs.to(device), targets.to(device)

            # Ensure targets have the right shape [batch_size, 1]
            if len(targets.shape) == 1:
                targets = targets.view(-1, 1)

            # Forward pass
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            # Record metrics
            losses.append(loss.item())
            mae = torch.mean(torch.abs(outputs - targets)).item()
            maes.append(mae)

        # Calculate average metrics for the epoch
        avg_loss = np.mean(losses)
        avg_mae = np.mean(maes)

        # Update history
        history['loss'].append(avg_loss)
        history['mae'].append(avg_mae)

        # Print epoch summary
        print(f"  Loss: {avg_loss:.4f} - MAE: {avg_mae:.4f}")

    return model, history

In [ ]:
def evaluate_model(region, frequency, model, test_loader, device, scaler_y=None):
    """Evaluate model performance on test set"""
    model = model.to(device)
    model.eval()

    all_preds = []
    all_targets = []

    with torch.no_grad():
        for inputs, targets in test_loader:
            # Move data to device
            inputs = inputs.to(device)

            # Forward pass
            outputs = model(inputs)

            # Move predictions back to CPU
            outputs = outputs.cpu().numpy()
            targets = targets.numpy()

            all_preds.extend(outputs)
            all_targets.extend(targets)

    # Convert lists to numpy arrays
    y_pred = np.array(all_preds)
    y_test = np.array(all_targets)

    # Inverse transform predictions and actual values
    if scaler_y is not None:
        # Reshape if needed
        y_pred_reshaped = y_pred.reshape(-1, 1)
        y_test_reshaped = y_test.reshape(-1, 1)

        y_pred = scaler_y.inverse_transform(y_pred_reshaped).flatten()
        y_test = scaler_y.inverse_transform(y_test_reshaped).flatten()

    # Calculate metrics
    mae = np.mean(np.abs(y_test - y_pred))
    rmse = np.sqrt(np.mean((y_test - y_pred)**2))
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

    print(f"Test MAE: {mae:.2f}")
    print(f"Test RMSE: {rmse:.2f}")
    print(f"Test MAPE: {mape:.2f}%")

    # Plot actual vs predicted
    plt.figure(figsize=(12, 6))
    plt.plot(y_test[:100], label='Actual')
    plt.plot(y_pred[:100], label='Predicted')
    plt.legend()
    plt.title(f'Actual vs Predicted Energy Demand for {region}')
    plt.ylabel('Energy Demand')
    plt.xlabel('Time Steps')
    plt.tight_layout()
    plt.savefig(f'results/lstm_actual_vs_predicted_{region}.png')
    plt.close()

    return mae, rmse, mape, y_pred

# 7. Forecasting Functions

In [ ]:
def recursive_forecast_without_weather(model, initial_input, scaler_X, scaler_y, n_steps=168):
    """Make recursive forecasts without using weather data.

    Args:
        model: Trained LSTM-Attention model
        initial_input: Last window of historical data
        scaler_X: Feature scaler
        scaler_y: Target scaler
        n_steps: Number of future steps to predict

    Returns:
        Numpy array of predictions
    """
    model.eval()

    # Scale the initial input
    n_samples, n_timesteps, n_features = initial_input.shape
    X_reshaped = initial_input.reshape(n_samples * n_timesteps, n_features)
    X_scaled = scaler_X.transform(X_reshaped).reshape(n_samples, n_timesteps, n_features)

    # Convert to tensor
    current_input = torch.tensor(X_scaled, dtype=torch.float32).to(device)

    # Storage for predictions
    predictions = []

    # Make a copy of the input that we'll update
    forecast_input = current_input.clone()
    
    print("Sample values from first timestep:")
    print(forecast_input[0, 0, :].cpu().numpy())


    with torch.no_grad():
        for i in range(n_steps):
            # Make one-step prediction
            output = model(forecast_input)

            # Convert prediction back to CPU and numpy
            pred = output.cpu().numpy()[0, 0]
            predictions.append(pred)

            # Create a new timestep by copying the most recent one and update with prediction
            new_timestep = forecast_input[:, -1:, :].clone()

            # Find the feature index that corresponds to the target value
            demand_indices = 0

            # If no demand indices found, use the first index as default
            target_idx = demand_indices[0] if demand_indices else 0

            # Update the demand feature with our prediction
            new_timestep[0, 0, target_idx] = torch.tensor(pred, dtype=torch.float32).to(device)

            # Shift window and add new timestep
            forecast_input = torch.cat([
                forecast_input[:, 1:, :],  # Remove oldest timestep
                new_timestep  # Add new timestep
            ], dim=1)

    # Inverse transform the predictions
    predictions = np.array(predictions).reshape(-1, 1)
    predictions = scaler_y.inverse_transform(predictions).flatten()

    return predictions

def recursive_forecast_with_weather(model, initial_input, scaler_X, scaler_y, feature_columns,
                                  forecast_weather, n_steps=168):
  """Make recursive forecasts using forecasted weather data.

  Args:
      model: Trained LSTM-Attention model
      initial_input: Last window of historical data
      scaler_X: Feature scaler
      scaler_y: Target scaler
      feature_columns: List of feature names
      forecast_weather: DataFrame with forecasted weather
      n_steps: Number of future steps to predict

  Returns:
      Numpy array of predictions
  """
  model.eval()

  # Scale the initial input
  n_samples, n_timesteps, n_features = initial_input.shape
  X_reshaped = initial_input.reshape(n_samples * n_timesteps, n_features)
  X_scaled = scaler_X.transform(X_reshaped).reshape(n_samples, n_timesteps, n_features)

  # Convert to tensor
  current_input = torch.tensor(X_scaled, dtype=torch.float32).to(device)

  # Storage for predictions
  predictions = []

  # Make a copy of the input that we'll update
  forecast_input = current_input.clone()

  with torch.no_grad():
      for i in range(n_steps):
          # Make one-step prediction
          output = model(forecast_input)

          # Convert prediction back to CPU and numpy
          pred = output.cpu().numpy()[0, 0]
          predictions.append(pred)

          # Create a new tensor for the next timestep
          new_timestep = torch.zeros(1, 1, n_features).to(device)

          # Get the weather data for this timestep
          weather_data = forecast_weather.iloc[i].copy()

          # Create array to hold new feature values
          new_features = np.zeros(n_features)

          # Fill in feature values based on weather and predicted demand
          for j, feature_name in enumerate(feature_columns):
              if feature_name in weather_data.index:
                  # This is a weather feature
                  new_features[j] = weather_data[feature_name]
              elif 'demand' in feature_name.lower():
                  # This is a demand feature, use the prediction
                  if '_lag_' in feature_name:
                      # This is a lagged demand feature, handle according to lag
                      lag = int(feature_name.split('_lag_')[1])
                      if i >= lag:
                          # Use an earlier prediction
                          new_features[j] = predictions[i - lag]
                      else:
                          # Use the initial input data for earlier lags
                          orig_idx = n_timesteps - lag + i
                          if orig_idx >= 0:
                              demand_idx = j % n_features
                              new_features[j] = initial_input[0, orig_idx, demand_idx]
                  else:
                      # Current demand
                      new_features[j] = pred
              elif 'day_of_week' in feature_name:
                  # Calculate day of week
                  new_features[j] = (forecast_weather.iloc[i]['day_of_week'])
              elif 'month' in feature_name:
                  # Month
                  new_features[j] = (forecast_weather.iloc[i]['month'])
              elif 'day_of_year' in feature_name:
                  # Day of year
                  new_features[j] = (forecast_weather.iloc[i]['day_of_year'])
              elif 'is_holiday' in feature_name:
                  # Holiday flag
                  new_features[j] = (forecast_weather.iloc[i]['is_holiday'])
              elif 'is_weekend' in feature_name:
                  # Weekend flag
                  new_features[j] = (forecast_weather.iloc[i]['is_weekend'])

          # Scale the new features
          new_features_scaled = scaler_X.transform(new_features.reshape(1, -1))

          # Update the new timestep tensor
          new_timestep[0, 0, :] = torch.tensor(new_features_scaled.flatten(), dtype=torch.float32).to(device)

          # Shift window and add new timestep
          forecast_input = torch.cat([
              forecast_input[:, 1:, :],  # Remove oldest timestep
              new_timestep  # Add new timestep
          ], dim=1)

  # Inverse transform the predictions
  predictions = np.array(predictions).reshape(-1, 1)
  predictions = scaler_y.inverse_transform(predictions).flatten()

  return predictions

# 8. Model Training and Evaluation

In [ ]:
def forecast_energy_demand(region, region_data, target_column, window_size,
                           epochs, batch_size, model, scaler_X, scaler_y,
                           feature_columns, use_weather=True):
    """Main function to forecast energy demand"""
    print(f"Forecasting energy demand for {region}...")

    # Get the last window from historical data for initial forecast input
    last_window_data = region_data.iloc[-window_size:].copy()

    last_date = region_data['datetime'].iloc[-1]
    print(f"Last date in historical data: {last_date}")
    print(f"First date in forecast period: {forecast_climate['datetime'].iloc[0]}")

    # Extract features for the last window
    exclude_cols = ['datetime', 'date', target_column, 'region']
    numeric_columns = region_data.select_dtypes(include=['number']).columns
    feature_columns = [col for col in numeric_columns if col not in exclude_cols]
    X = last_window_data[feature_columns].values.reshape(1, window_size, len(feature_columns))

    # Make recursive forecasts
    n_forecast_hours = 168  # One week

    # if use_weather and region == "Toronto":
    if use_weather:
        print("Using weather data for forecasting...")
        predictions = recursive_forecast_with_weather(
            model, X, scaler_X, scaler_y, feature_columns,
            forecast_climate, n_steps=n_forecast_hours
        )
    else:
        print("Not using weather data for forecasting...")
        predictions = recursive_forecast_without_weather(
            model, X, scaler_X, scaler_y, n_steps=n_forecast_hours
        )

    # Create forecast dates
    forecast_dates = [last_date + timedelta(hours=i+1) for i in range(n_forecast_hours)]

    # Create forecast DataFrame
    forecast_df = pd.DataFrame({
        'datetime': forecast_dates,
        'forecast': predictions,
        'region': region
    })

    # Merge with actual demand for evaluation if available
    try:
        if region == "Toronto":
            eval_df = pd.merge(
                forecast_df,
                actual_toronto_demand[['datetime', "actual"]],
                on='datetime',
                how='inner'
            )
        else:
            eval_df = pd.merge(
                forecast_df,
                actual_ontario_demand[['datetime', "actual"]],
                on='datetime',
                how='inner'
            )

        # Calculate evaluation metrics
        mae = np.mean(np.abs(eval_df['actual'] - eval_df['forecast']))
        rmse = np.sqrt(np.mean((eval_df['actual'] - eval_df['forecast'])**2))
        mape = np.mean(np.abs((eval_df['actual'] - eval_df['forecast']) / eval_df['actual'])) * 100

        print(f"Evaluation Results for {region}:")
        print(f"MAE: {mae:.2f}")
        print(f"RMSE: {rmse:.2f}")
        print(f"MAPE: {mape:.2f}%")

        # Plot forecasts vs actual
        plt.figure(figsize=(12, 6))

        # Plot historical data
        historical_dates = region_data['datetime'].iloc[-168:].values
        historical_values = region_data[target_column].iloc[-168:].values
        plt.plot(historical_dates, historical_values, label='Historical', color='blue')

        # Plot forecasted data
        plt.plot(eval_df['datetime'], eval_df['forecast'], label='LSTM-Attention Forecast', color='red')

        # Plot actual data
        plt.plot(eval_df['datetime'], eval_df['actual'], label='Actual', color='green')

        weather_text = "with weather" if use_weather else "without weather"
        plt.title(f'Energy Demand Forecast vs Actual for {region} ({weather_text})\nMAE: {mae:.2f}, RMSE: {rmse:.2f}, MAPE: {mape:.2f}%')
        plt.xlabel('Date')
        plt.ylabel('Energy Demand')
        plt.legend()
        plt.grid(True)
        plt.xticks(rotation=45)
        plt.tight_layout()

        # Save the plot
        weather_label = "with_weather" if use_weather else "without_weather"
        plt.savefig(f'results/lstm_{region}_{weather_label}_forecast_vs_actual.png')
        plt.close()

        # Create results DataFrame
        results = pd.DataFrame({'MAE': [mae], "RMSE": [rmse], "MAPE": [mape]})
        results.to_csv(f'results/lstm_{region}_{weather_label}_metrics.csv', index=False)

    except Exception as e:
        print(f"Could not perform evaluation against actual data: {e}")
        eval_df = None

    # Save forecast DataFrame
    weather_label = "with_weather" if use_weather else "without_weather"
    forecast_df.to_csv(f'results/lstm_{region}_{weather_label}_forecast.csv', index=False)

    print(f"Forecast complete for {region}.")

    return forecast_df, model, scaler_X, scaler_y

# 9. Model Execution

In [ ]:
# Parameters for the model
region = "Ontario"
window_size = 168  # One week of hourly data
epochs = 50
batch_size = 16
target_column = 'zonal_demand' # what we are aiming to predict, for toronto it's zonal_demand

# Create directories for results
os.makedirs('results', exist_ok=True)
os.makedirs('models', exist_ok=True)

# Run forecast 
print("Starting energy demand forecast")
data = hourly_ontario.copy() # substitute with toronto data here depending on what you're looking to run

# we should split up training and the other steps, or else we're going to run into the same compute
# issues as before
model_save_path = f"models/lstm_{region}_model.pth"
# Create cache directory if it doesn't exist
os.makedirs("cache", exist_ok=True)
os.makedirs("models", exist_ok=True)

# Prepare data
X, y = prepare_data_for_model(region, data, target_column, window_size)

# Scale features
n_samples, n_timesteps, n_features = X.shape
X_reshaped = X.reshape(n_samples * n_timesteps, n_features)

scaler_X = StandardScaler()
scaler_X.fit(X_reshaped)
X_scaled = scaler_X.transform(X_reshaped).reshape(n_samples, n_timesteps, n_features)

scaler_y = StandardScaler()
scaler_y.fit(y)
y_scaled = scaler_y.transform(y).flatten()

# Create dataloaders
dataset = TimeSeriesDataset(X_scaled, y_scaled)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Initialize model
input_shape = (X_scaled.shape[1], X_scaled.shape[2])  # (timesteps, features)
model = LSTMAttentionModel(
    input_shape=input_shape,
    hidden_dim=128,
    lstm_layers=2,
    dropout=0.3
)

# Train model
print(f"Training LSTM-Attention model for {region}...")
model, history = train_model(
    model,
    loader,
    device,
    epochs=50,
    learning_rate=0.001,
    model_save_path=model_save_path
)

exclude_cols = ['datetime', 'date', target_column, 'region']
numeric_columns = data.select_dtypes(include=['number']).columns
feature_columns = [col for col in numeric_columns if col not in exclude_cols]

In [ ]:
ontario_forecast_with_weather, ontario_model, ontario_scaler_X, ontario_scaler_y = forecast_energy_demand(
    region='Ontario',
    region_data=data,
    target_column=target_column_ontario,
    window_size=window_size,
    epochs=epochs,
    batch_size=batch_size,
    model=model,
    scaler_X=scaler_X,
    scaler_y=scaler_y,
    feature_columns=feature_columns,
    use_weather=True
)

In [ ]:
ontario_forecast_with_weather, ontario_model, ontario_scaler_X, ontario_scaler_y = forecast_energy_demand(
    region='Ontario',
    region_data=data,
    target_column=target_column,
    window_size=window_size,
    epochs=epochs,
    batch_size=batch_size,
    model=model,
    scaler_X=scaler_X,
    scaler_y=scaler_y,
    feature_columns=feature_columns,
    use_weather=False
)

# 11. Visualize Predicted vs. Actual Demand

In [ ]:
# Create combined visualization of all predictions vs actual
try:
    # Read the prediction results
    ontario_with_weather = pd.read_csv('results/lstm_Ontario_with_weather_forecast.csv')
    ontario_without_weather = pd.read_csv('results/lstm_Ontario_without_weather_forecast.csv')

    # Convert datetime columns
    ontario_with_weather['datetime'] = pd.to_datetime(ontario_with_weather['datetime'])
    ontario_without_weather['datetime'] = pd.to_datetime(ontario_without_weather['datetime'])

    # Prepare actual data
    actual_ontario = actual_ontario_demand.copy()
    actual_ontario['datetime'] = pd.to_datetime(actual_ontario['datetime'])

    # Create the plot
    plt.figure(figsize=(15, 10))

    # Toronto plot
    # plt.subplot(2, 1, 1)
    plt.plot(ontario_with_weather['datetime'], ontario_with_weather['forecast'],
             label='LSTM-Attention predictions with weather forecast', color='red')
    plt.plot(ontario_without_weather['datetime'], ontario_without_weather['forecast'],
             label='LSTM-Attention without weather forecast', color='blue', linestyle='--')
    plt.plot(actual_ontario['datetime'], actual_ontario['actual'],
             label='Actual demand', color='green')

    plt.title('Ontario Energy Demand Forecasting Comparison')
    plt.xlabel('Date')
    plt.ylabel('Energy Demand')
    plt.legend()
    plt.grid(True)

    plt.savefig('results/lstm_combined_forecast_comparison.png')
    plt.show()

except Exception as e:
    print(f"Could not create visualization: {e}")